# Convolutional Neural Networks

[LeNet-5](http://yann.lecun.com/exdb/lenet/)

Uses [Lasagne](https://github.com/Lasagne/Lasagne) and [nolearn](https://github.com/dnouri/nolearn).

In [26]:
import os

import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

In [27]:
def load2d(path, size=32, nband=5):
    # don't use QSO for now
    files = [os.path.join(path, f) for f in os.listdir(path)
             if f.split(".")[0] == "GALAXY" or f.split(".")[0] == "STAR"]
    
    X = np.zeros((len(files), nband, size, size))
    y = np.zeros((len(files)))
    
    for i, f in enumerate(files):
        npy = np.load(f)
        
        X[i, :, :, :] = npy

        y[i] = np.array([-1 if "GALAXY" in f else 1])

    X = (X.max() - X) / (X.max() - X.min())
    
    X, y = shuffle(X, y)

    return X, y

In [28]:
X, y = load2d("result", size=48)  # load 2-d data
print(X.shape)
print(y.shape)

(253, 5, 48, 48)
(253,)


In [34]:
net2 = NeuralNet(
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('conv3', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),
        ('hidden4', layers.DenseLayer),
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 5, 48, 48),
    conv1_num_filters=32, conv1_filter_size=(3, 3), pool1_pool_size=(2, 2),
    conv2_num_filters=64, conv2_filter_size=(2, 2), pool2_pool_size=(2, 2),
    conv3_num_filters=128, conv3_filter_size=(2, 2), pool3_pool_size=(2, 2),
    hidden4_num_units=500,
    hidden5_num_units=500,
    output_num_units=1, output_nonlinearity=None,

    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=True,
    max_epochs=500,
    verbose=1,
    )

net2.fit(X[:150, :, :, :], y[:150])

# We'll pickle the trained model so that we can load it back later:
#import cPickle as pickle
#with open('net2.pickle', 'wb') as f:
#    pickle.dump(net2, f, -1)

# Neural Network with 1894125 learnable parameters

## Layer information

  #  name     size
---  -------  ---------
  0  input    5x48x48
  1  conv1    32x46x46
  2  pool1    32x23x23
  3  conv2    64x22x22
  4  pool2    64x11x11
  5  conv3    128x10x10
  6  pool3    128x5x5
  7  hidden4  500
  8  hidden5  500
  9  output   1

  epoch    train loss    valid loss    train/val  dur
-------  ------------  ------------  -----------  -----
      1       0.98248       0.92848      1.05816  5.37s
      2       0.89990       0.87865      1.02418  5.95s
      3       0.82167       0.84357      0.97404  8.03s
      4       0.75352       0.83430      0.90318  7.56s
      5       0.70736       0.85704      0.82535  5.22s
      6       0.69324       0.89708      0.77278  5.41s
      7       0.70427       0.92420      0.76204  5.36s
      8       0.71690       0.92213      0.77744  5.46s
      9       0.71544       0.90109      0.79397  5.35s
     10       0.70478       0.87708      0.80355  5.46s


NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7f6735ba21d0>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x7f6735ba2160>,
     conv1_filter_size=(3, 3), conv1_num_filters=32,
     conv2_filter_size=(2, 2), conv2_num_filters=64,
     conv3_filter_size=(2, 2), conv3_num_filters=128, custom_score=None,
     hidden4_num_units=500, hidden5_num_units=500,
     input_shape=(None, 5, 48, 48),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('conv1', <class 'lasagne.layers.conv.Conv2DLayer'>), ('pool1', <class 'lasagne.layers.pool.MaxPool2DLayer'>), ('conv2', <class 'lasagne.layers.conv.Conv2DLayer'>), ('pool2', <class 'lasagne.layers.pool.MaxPool2DLayer'>), ('conv3', <class..., <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=500, more_params={},
     objective=<function objective at 0x7f6735ba0510>,
     objective

In [35]:
pred = net2.predict(X[150:, :, :, :])
truth = y[150:]

In [36]:
pred2 = [1 if p >= 0 else -1 for p in pred]

In [37]:
print("Accuracy: {0:.3f}".format((pred2 == truth).sum() / len(truth)))

Accuracy: 0.971


In [38]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(truth, pred)
print("Area under ROC curve: {0:.3f}".format(auc))

Area under ROC curve: 0.989


# TODO

- Compare with TPC
- Use magnitudes? Colors? Both?
- Hyperparameter search
- Data augmentation: crop, rotate, flip, etc.
- Dropout layers
- CFHTLenS, KiDS
- Include QSOs?